<a href="https://colab.research.google.com/github/DMS-999/ML-Handbook-materials/blob/main/Simulative_%D0%9F%D0%BE%D0%B4%D0%B1%D0%BE%D1%80%D0%BA%D0%B0_%D1%81%D0%BE%D0%B2%D0%B5%D1%82%D0%BE%D0%B2_%D0%BF%D0%BE_%D1%83%D0%BB%D1%83%D1%87%D1%88%D0%B5%D0%BD%D0%B8%D1%8E_%D0%BA%D0%BE%D0%B4%D0%B0_%D0%B2_Python_2_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В программировании важно заботиться о безопасности кода, в том числе о сокрытии приватных методов, об обработке возможных ошибок и о написании кода, не предполагающего возникновения ошибки

# 1. Защита от значений None

Ниже приведен обычный кусок кода, где идет обращение ко вложенному атрибуту

In [ ]:
class Owner:
  def __init__(self, name):
    self.name = name

class Dog:
  def __init__(self, owner=None):
    self.owner = owner

In [ ]:
owner = Owner("Bob")
dog = Dog(owner)

In [ ]:
if dog.owner.name == "Bob":
  print("this is bob's dog")

this is bob's dog


Однако данный кусок кода вряд ли будет одобрен для добавления в промышленную кодовую базу по причине того, что здесь возможна ситуация, когда какой-либо из атрибутов имеет значение None

Ниже приведен пример возможной обработки потенциальной ошибки. В данном случае отсутствующее значение атрибута не приведет к возникновению неисправной ситуации

In [ ]:
dog = Dog()

In [ ]:
if dog and dog.owner and dog.owner.name == "Bob":
  print("this is bob's dog")

Логические операторы в Python являются ленивыми, то есть они прекращают исполнение в случае, когда конечный результат не поменяется вне зависимости от значений, следующих далее

# 2. Защита от итерирования по None значениям

Ниже приведен пример обычного итерирования по списку

In [ ]:
my_list = [1, 2, 3, 4, 5]

In [ ]:
for item in my_list:
  print(item)

1
2
3
4
5


Однако в случае, если `mylist` равен None, мы получим ошибку

Подобную ситуацию можно обработать с помощью использования оператора or

In [ ]:
my_none_list = None

In [ ]:
for item in my_none_list or []:
  print(item)

# 3. Функции с нижним подчеркиванием

Ниже приведен пример класса, где метод `run` использует внутренние методы `clean` и `transform`

In [ ]:
class MyClass:
  def run(self, ls: list[str], n: int) -> list[str]:
    cleaned_ls = self.clean(ls)
    transformed_ls = self.transform(cleaned_ls, n)
    return transformed_ls

  def clean(ls: list[str]) -> list[str]:
    pass

  def transform(ls: list[str], n: int) -> list[str]:
    pass

В промышленном коде следует стараться писать как можно более явный код, где видна четкая грань между внешними, используемыми другими объектами, и внутренними, используемыми только самим объектом, методами класса. В данном случае внешние и внтуренние методы никак не отличаются, пользователь может вызвать любой из методов

Это можно исправить с помощью символа нижнего подчеркивания

In [ ]:
class MyClass:
  def run(self, ls: list[str], n: int) -> list[str]:
    cleaned_ls = self._clean(ls)
    transformed_ls = self._transform(cleaned_ls, n)
    return transformed_ls

  def _clean(ls: list[str]) -> list[str]:
    pass

  def _transform(ls: list[str], n: int) -> list[str]:
    pass

В функциональном плане разницы не будет. Эти методы также могут быть вызваны другими объектами, но в данном случае будет явно указано, что это внутренний метод и что лучше его не вызывать из-вне объекта

# 4. Использование декораторов для обработки ошибок и логирования

Вот класс с 3 функциями, и каждая функция выполняет разные действия. Однако обратите внимание, что в разных функциях есть похожие шаги — блок try-except, а также функциональность логирования.

In [ ]:
import logging
import sys

logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='%(message)s')

logger = logging.getLogger(__name__)

In [ ]:
import time

In [ ]:
class NewClass:
  def func1(self):
    try:
      time.sleep(1)
      print("hello fron func1")
      logger.info("func1")
    except Exception as e:
      logger.error(str(e))
  def func2(self):
    try:
      time.sleep(2)
      print("hello fron func2")
      logger.info("func2")
    except Exception as e:
      logger.error(str(e))
  def func3(self):
    try:
      time.sleep(3)
      print("hello fron func3")
      logger.info("func3")
    except Exception as e:
      logger.error(str(e))


In [ ]:
nc = NewClass()

nc.func1()
nc.func2()
nc.func3()

hello fron func1
hello fron func2
hello fron func3


Одной хорошей практикой для уменьшения количества повторяющегося кода является написание функции-декоратора, содержащей общую функциональность.

In [ ]:
def handle_exception_and_logging(func):
  def wrapper(*arg, **kwargs):
    try:
      res = func(*arg, **kwargs)
      logger.info(f"{func.__name__}")
    except Exception as e:
      logger.error(str(e))
  return wrapper

In [ ]:
class NewClass:
  @handle_exception_and_logging
  def func1(self):
      time.sleep(1)
      print("hello fron func1")
  @handle_exception_and_logging
  def func2(self):
      time.sleep(2)
      print("hello fron func2")
  @handle_exception_and_logging
  def func3(self):
      time.sleep(3)
      print("hello fron func3")

In [ ]:
nc = NewClass()

nc.func1()
nc.func2()
nc.func3()

hello fron func1
hello fron func2
hello fron func3


Таким образом, если мы хотим обновить общий код (блок try-except и код логирования), нам больше не нужно обновлять его в 3 местах — нам нужно только обновить код нашего декоратора, содержащий общую функциональность.